In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)# OPTIONAL


import accelerate
from transformers import AutoModel, BertTokenizerFast
from datasets import Dataset

from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments, TrainerCallback
import os

os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""

In [ ]:
df=pd.read_csv("poems_topics_data.csv")
df.head()

,category,filename,poem_text
0,alone,AlonePoems06HaikuAlonePoembyjohntiongchunghoo.txt,loneliness\nhis childhood\nwarms him up\nlonel...
1,alone,AlonePoems1999AlonePoembyDeanaRode.txt,Sit in a chair\nalone\nPeople I thought were\n...
2,alone,AlonePoems2006AlonePoembyDeanaRode.txt,Tell me why it has to be this way\nwhy I must ...
3,alone,AlonePoemsADebtorToMercyAlonePoembyAugustusMon...,"A debtor to mercy alone, of covenant mercy I s..."
4,alone,AlonePoemsAgainLeftAlonePoembyRaviSathasivam.txt,A year has been passed since I left home\nAgai...


In [ ]:
len(df)

14335

In [ ]:
lyrics = df['poem_text'].tolist()
# lyrics

In [ ]:
def tokenize_function(examples):
    # Tokenize the lyrics
    tokenized = tokenizer(examples['poem_text'], padding='max_length', truncation=True, max_length=256)
    # For masked language modeling, we need to create labels by copying input_ids
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

In [ ]:
dataset = Dataset.from_pandas(df[['poem_text']])
# tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.3)  # 80% train, 20% validation
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
output_dir = './fine_tuned_bert_checkpoint_f1500'          # Directory for checkpoints
final_model_dir = './fine_tuned_bert_model_f1500'  # Directory for final saved model

In [ ]:
from transformers import AutoTokenizer, RobertaForMaskedLM
import torch

In [ ]:
if os.path.exists(final_model_dir):
    print(f"Loading fine-tuned model from {final_model_dir}")
    tokenizer = RobertaTokenizer.from_pretrained(final_model_dir)
    model = RobertaForMaskedLM.from_pretrained(final_model_dir)
else:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created checkpoint directory: {output_dir}")

    checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-') and os.path.isdir(os.path.join(output_dir, d))]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
        checkpoint_path = os.path.join(output_dir, latest_checkpoint)
        print(f"Loading latest checkpoint from {checkpoint_path}")
        tokenizer = RobertaTokenizer.from_pretrained(checkpoint_path)
        model = RobertaForMaskedLM.from_pretrained(checkpoint_path)
    else:
        print("No saved model or checkpoint found. Starting from pre-trained BERT.")
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForMaskedLM.from_pretrained('roberta-base')

    # Freeze all parameters initially
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the top 5 layers (7-11) and the output head
    num_layers = len(model.roberta.encoder.layer)  # 12 for bert-base
    layers_to_finetune = 4  # Change to 4 if you want layers 8-11
    for layer in model.roberta.encoder.layer[num_layers - layers_to_finetune:]:
        for param in layer.parameters():
            param.requires_grad = True

    # Unfreeze the MLM head (cls.predictions)
    for param in model.lm_head.parameters():
        param.requires_grad = True

    # Verify trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.2%})")

Created checkpoint directory: ./fine_tuned_bert_checkpoint_f1500
No saved model or checkpoint found. Starting from pre-trained BERT.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Trainable parameters: 67,597,401 / 124,697,433 (54.21%)


In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['poem_text'], padding='max_length', truncation=True, max_length=128)
    tokenized['labels'] = tokenized['input_ids'].copy()  # For MLM, labels are input_ids
    return tokenized

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10034 [00:00<?, ? examples/s]

Map:   0%|          | 0/4301 [00:00<?, ? examples/s]

In [ ]:
# Step 5: Define a custom metrics function for accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Get the predicted token IDs
    # In MLM, labels are the original input_ids, and -100 indicates non-masked tokens (ignored in loss)
    # Compare predictions and labels where labels != -100 (masked positions)
    mask = labels != -100
    accuracy = np.mean(predictions[mask] == labels[mask])
    return {"accuracy": accuracy}

In [ ]:
import transformers

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
import numpy as np

In [ ]:
from transformers import DataCollatorForWholeWordMask

In [ ]:
from transformers import Trainer, TrainingArguments
import os
import torch

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,  # Explicitly set for evaluation later
    logging_dir='./logs',
    logging_steps=500,
    fp16=True,  # Optional: Reduce memory usage during training
)

data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Set environment variable to avoid memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Clear GPU memory before training
torch.cuda.empty_cache()

# Train the model
trainer.train(resume_from_checkpoint=os.path.exists(output_dir) and any(d.startswith('checkpoint-') for d in os.listdir(output_dir)))

# Evaluate after training
print("Training completed. Running evaluation...")
torch.cuda.empty_cache()  # Clear memory before evaluation
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: damanimalay (damanimalay-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Step,Training Loss
500,2.636000
1000,2.503600
1500,2.411900
2000,2.389700
2500,2.337300
3000,2.273000
3500,2.260600
4000,2.228700
4500,2.201100
5000,2.214500


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:1286: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:1286: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:1286: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:1286: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(
/usr/loc

Training completed. Running evaluation...


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:1286: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.60 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.29 GiB is free. Process 6174 has 10.45 GiB memory in use. Of the allocated memory 5.59 GiB is allocated by PyTorch, and 4.72 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def predict_masked_text(input_text, top_k=5):
    model.eval()
    if "[MASK]" in input_text:
        input_text = input_text.replace("[MASK]", "<mask>")
        print("Note: Replaced [MASK] with <mask>.")

    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    mask_token_id = tokenizer.mask_token_id
    mask_positions = (input_ids == mask_token_id).nonzero(as_tuple=True)[1]

    if mask_positions.numel() == 0:
        raise ValueError("No <mask> token found in input text. Use '<mask>'.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    model.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    results = {}
    for mask_position in mask_positions:
        mask_position = mask_position.item()
        mask_logits = logits[0, mask_position]
        probabilities = torch.softmax(mask_logits, dim=-1)
        top_k_probs, top_k_ids = torch.topk(probabilities, top_k)
        top_k_tokens = [tokenizer.decode([id.item()], skip_special_tokens=True) for id in top_k_ids]
        results[mask_position] = list(zip(top_k_tokens, top_k_probs.tolist()))

    return results

# Example with multiple masks
text = "Loneliness is <mask> for me and I am <mask> of this <mask>"
predictions = predict_masked_text(text, top_k=30)
for pos, preds in predictions.items():
    print(f"\nMask at position {pos}:")
    for word, prob in preds:
        print(f"  {word}: {prob:.4f}")


Mask at position 4:
   not: 0.1687
   life: 0.1293
   new: 0.1031
   hard: 0.0491
   bad: 0.0472
   good: 0.0365
   here: 0.0209
   difficult: 0.0171
   different: 0.0145
   there: 0.0144
   coming: 0.0138
   great: 0.0115
   normal: 0.0102
   real: 0.0101
   enough: 0.0089
   unhealthy: 0.0089
   nothing: 0.0083
   rare: 0.0083
   unusual: 0.0072
   easy: 0.0070
   wrong: 0.0068
   only: 0.0064
   unbearable: 0.0062
   impossible: 0.0058
   painful: 0.0057
   growing: 0.0054
   something: 0.0049
   worse: 0.0045
   huge: 0.0043
   uncomfortable: 0.0043

Mask at position 10:
   proud: 0.3318
   aware: 0.3216
   tired: 0.0764
   ashamed: 0.0752
   glad: 0.0533
   conscious: 0.0313
   sick: 0.0281
   afraid: 0.0204
   weary: 0.0068
   sorry: 0.0062
   scared: 0.0055
   unaware: 0.0035
   certain: 0.0034
   terrified: 0.0029
   reminded: 0.0026
   guilty: 0.0020
   because: 0.0017
   full: 0.0016
   grateful: 0.0014
   sad: 0.0013
   fearful: 0.0012
   thankful: 0.0012
   sure: 0.0011
  

In [ ]:
final_model_dir = './fine_tuned_roberta_model_f1500'
trainer.save_model(final_model_dir)
tokenizer.save_pretrained(final_model_dir)

('./fine_tuned_roberta_model_f1500/tokenizer_config.json',
 './fine_tuned_roberta_model_f1500/special_tokens_map.json',
 './fine_tuned_roberta_model_f1500/vocab.json',
 './fine_tuned_roberta_model_f1500/merges.txt',
 './fine_tuned_roberta_model_f1500/added_tokens.json')

In [ ]:
!zip -r fine_tuned_roberta_model_f1500.zip fine_tuned_roberta_model_f1500

  adding: fine_tuned_roberta_model_f1500/ (stored 0%)
  adding: fine_tuned_roberta_model_f1500/vocab.json (deflated 68%)
  adding: fine_tuned_roberta_model_f1500/model.safetensors (deflated 23%)
  adding: fine_tuned_roberta_model_f1500/tokenizer_config.json (deflated 76%)
  adding: fine_tuned_roberta_model_f1500/training_args.bin (deflated 52%)
  adding: fine_tuned_roberta_model_f1500/merges.txt (deflated 53%)
  adding: fine_tuned_roberta_model_f1500/special_tokens_map.json (deflated 84%)
  adding: fine_tuned_roberta_model_f1500/config.json (deflated 49%)


In [ ]:
!zip -r fine_tuned_roberta_keras.zip fine_tuned_roberta_keras

  adding: fine_tuned_roberta_keras/ (stored 0%)
  adding: fine_tuned_roberta_keras/vocab.json (deflated 68%)
  adding: fine_tuned_roberta_keras/tokenizer_config.json (deflated 76%)
  adding: fine_tuned_roberta_keras/tf_model.h5 (deflated 19%)
  adding: fine_tuned_roberta_keras/merges.txt (deflated 53%)
  adding: fine_tuned_roberta_keras/special_tokens_map.json (deflated 84%)
  adding: fine_tuned_roberta_keras/config.json (deflated 49%)


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, TFRobertaForMaskedLM

In [ ]:
tf_model = TFRobertaForMaskedLM.from_pretrained(final_model_dir, from_pt=True)

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
final_keras_dir = './fine_tuned_roberta_keras'
tf_model.save_pretrained(final_keras_dir)  # Saves as TensorFlow weights
tokenizer.save_pretrained(final_keras_dir)  # Saves tokenizer separately

('./fine_tuned_roberta_keras/tokenizer_config.json',
 './fine_tuned_roberta_keras/special_tokens_map.json',
 './fine_tuned_roberta_keras/vocab.json',
 './fine_tuned_roberta_keras/merges.txt',
 './fine_tuned_roberta_keras/added_tokens.json')

In [ ]:
tf_model.save(os.path.join(final_keras_dir, 'fine_tuned_roberta_model_f1500.h5'), save_format='h5')

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.

In [ ]:
test_texts = [
    "Roses are red\nViolets are [MASK]",
    "The sky is [MASK]\nThe grass is green",
    "I love to [MASK]\non weekends"
]

for text in test_texts:
    print(f"\nInput: {text}")
    predictions = predict_masked_text(text, top_k=5)
    print("Predictions:")
    for word, prob in predictions:
        print(f"  {word}: {prob:.4f}")


Input: Roses are red
Violets are [MASK]


RuntimeError: a Tensor with 0 elements cannot be converted to Scalar

In [ ]:
tokenizer_keras = RobertaTokenizer.from_pretrained(final_keras_dir)
tf_model_keras = TFRobertaForMaskedLM.from_pretrained(final_keras_dir)

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at ./fine_tuned_roberta_keras.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [ ]:
def predict_masked_text_tf(input_text, top_k=5):
    if "[MASK]" in input_text:
        input_text = input_text.replace("[MASK]", "<mask>")
    inputs = tokenizer(input_text, return_tensors="tf")
    outputs = tf_model(inputs)
    logits = outputs.logits  # Shape: (1, seq_length, vocab_size)
    mask_position = tf.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0][0]
    mask_logits = logits[0, mask_position]
    probabilities = tf.nn.softmax(mask_logits)
    top_k_ids = tf.nn.top_k(probabilities, k=top_k).indices
    top_k_probs = tf.nn.top_k(probabilities, k=top_k).values
    top_k_tokens = [tokenizer.decode([id], skip_special_tokens=True) for id in top_k_ids.numpy()]
    return list(zip(top_k_tokens, top_k_probs.numpy()))

In [ ]:
import tensorflow as tf

In [ ]:
# Test it
test_text = "It is so <mask> that we have to deal with all this"
predictions = predict_masked_text_tf(test_text, top_k=5)
print(f"Input: {test_text}")
for word, prob in predictions:
    print(f"  {word}: {prob:.4f}")

Input: It is so <mask> that we have to deal with all this
   unfair: 0.3835
   sad: 0.2777
   bad: 0.0298
   awful: 0.0236
   crazy: 0.0182
